# Homework 3
1. Добавить фильтрацию уже просмотренных айтемов при получении рекомендаций через ANN.
2. Попробовать добиться лучшего перформанса, изменяя параметры ANN индекса.
3. Попробовать любую другую библиотеку для ANN, сравнить результаты.
4. Лучшее решение засабмиттить в сореву на ODS.

# Imports

In [1]:
# turn off bad autocompleter
%config Completer.use_jedi = False
# magic for modules reloading 
%load_ext autoreload
%autoreload 2

In [74]:
import pandas as pd
import numpy as np

from copy import deepcopy
from tqdm import tqdm
# from scipy.stats import mode
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import gc
# import warnings
# warnings.filterwarnings("ignore")

from itertools import product
from scipy.sparse import csr_matrix

from rectools import Columns
from rectools.dataset import Dataset, Interactions
from rectools.metrics import MAP

import hnswlib
import nmslib
from implicit.als import AlternatingLeastSquares

In [126]:
def filter_seen(recos, user_ids=test_users):
    reco_filtered = []
    n_to_add_history = []
    for user_id, user_recos in tqdm(zip(user_ids, recos), total=len(user_ids)):
        seen_u = seen_items.get_group(user_id).values
        new_recos = user_recos[~np.in1d(user_recos, seen_u)]
        new_recos = new_recos[:10]
        n_to_add = 10 - len(new_recos)
        n_to_add_history.append(n_to_add)
        if n_to_add:
            popular_new = popular_items[~popular_items.isin(new_recos)].values[-n_to_add]
            new_recos = np.append(new_recos, popular_new)
        reco_filtered.append(new_recos)
    return np.asarray(reco_filtered)

In [26]:
def calc_map(reco):
    global interactions_test, test_users, n_recs
    reco_df = pd.DataFrame(
        {
            Columns.User: np.repeat(test_users, n_recs),
            Columns.Item: np.asarray(reco).ravel(),
            Columns.Rank: np.tile(np.arange(n_recs) + 1, len(test_users)),
        }
    )
    metric = MAP(k=n_recs).calc(reco_df, interactions_test)
    return metric

# Load data
Обработка данных такая же как и в ДЗ 1. Поэтому часть клеток я опущу для краткости

In [5]:
interactions = pd.read_csv('data_full/interactions.csv')
# users = pd.read_csv('data_full/users.csv')
# items = pd.read_csv('data_full/items.csv')

In [6]:
interactions = interactions.rename(columns = {
    'last_watch_dt': Columns.Datetime,
    'total_dur': Columns.Weight,
})

In [7]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime])

In [8]:
interactions_train = interactions.query("datetime >= '2021-07-15' and datetime < '2021-08-16'")
interactions_test = interactions.query("datetime >= '2021-08-16'")

In [9]:
seen_items = interactions_train.groupby('user_id')['item_id']

In [10]:
popular_items = interactions_train.item_id.value_counts().sort_values()

In [11]:
interactions_train.shape, interactions_train[["user_id", "item_id"]].nunique()

((1683311, 5),
 user_id    423800
 item_id     10185
 dtype: int64)

In [12]:
interactions_test.shape, interactions_test[["user_id", "item_id"]].nunique()

((424436, 5),
 user_id    167348
 item_id      7106
 dtype: int64)

Будем работать только с "горячими" юзерами и айтемами

In [13]:
interactions_test = interactions_test.loc[
    interactions_test["user_id"].isin(interactions_train["user_id"])
    & interactions_test["item_id"].isin(interactions_train["item_id"])
]

In [14]:
interactions_test.shape, interactions_test[["user_id", "item_id"]].nunique()

((271586, 5),
 user_id    95326
 item_id     6331
 dtype: int64)

In [15]:
dataset_train = Dataset.construct(interactions_train)
dataset_test = Dataset.construct(interactions_test)

In [16]:
# ui_csr = dataset_train.get_user_item_matrix()

In [17]:
ui_csr = csr_matrix(
    (
        np.ones(len(interactions_train), dtype=np.float32),
        (
            interactions_train["user_id"].values,
            interactions_train["item_id"].values,
        ),
    ),
)

In [18]:
iu_csr = ui_csr.T.tocsr(copy=False)  # В версии implicit==0.4.4 матрицу нужно транспонировать

# ALS

In [20]:
%%time
model = AlternatingLeastSquares(factors=32, iterations=10, num_threads=12)  # Если есть GPU, можно на нем
model.fit(iu_csr, show_progress=True)

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 1min 11s, sys: 12 s, total: 1min 24s
Wall time: 1min 2s


In [21]:
users_factors = model.user_factors
items_factors = model.item_factors

In [22]:
test_users = interactions_test["user_id"].unique()
test_items = interactions_test['item_id'].unique()

In [23]:
# test_user_factors = users_factors[np.where(np.in1d(dataset_train.user_id_map.external_ids, test_users))[0]]
test_user_factors = users_factors[test_users]

In [24]:
%%time
n_recs = 10
reco_recommend = []
for user_id in tqdm(test_users):
# for user_id in tqdm(dataset_test.user_id_map.internal_ids):
    
    res = model.recommend(user_id, ui_csr, N=n_recs, filter_already_liked_items=False)
    recs = [tup[0] for tup in res]
    reco_recommend.append(recs)
    
reco_recommend = np.array(reco_recommend)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95326/95326 [01:06<00:00, 1437.87it/s]


CPU times: user 1min 9s, sys: 7.29 s, total: 1min 16s
Wall time: 1min 6s


In [25]:
%%time
reco_direct = []
for user_id in tqdm(test_users):
# for user_id in tqdm(dataset_test.user_id_map.internal_ids):
    user_factors = users_factors[user_id]
    scores = items_factors @ user_factors
    unsorted_reco_positions = scores.argpartition(-n_recs)[-n_recs:]
    unsorted_reco_scores = scores[unsorted_reco_positions]
    recs = unsorted_reco_positions[unsorted_reco_scores.argsort()[::-1]]
    reco_direct.append(recs)
    
reco_direct = np.array(reco_direct)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95326/95326 [01:03<00:00, 1495.00it/s]


CPU times: user 1min 5s, sys: 6.58 s, total: 1min 11s
Wall time: 1min 3s


In [61]:
calc_map(reco_recommend)

0.015689577893156493

In [64]:
calc_map(reco_direct)

0.015689577893156493

# ANN

## nmslib

In [29]:
# Set index parameters
# These are the most important onese

M = 200 # For a newly added point find this number of most closest points
# that make the initial neighborhood of the point. When more
# points are added, this neighborhood may be expanded.

efC = 100 # The depth of the search that is used to find neighbors during indexing.

num_threads = 2
index_time_params = {
    'M': M, 
    'indexThreadQty': num_threads, 
    'efConstruction': efC, 
#     'post' : 0,
#     'skip_optimized_index' : 1 # using non-optimized index!
}

space_name='l2' # l2, ip, cosinesimil
# space_name='cosinesimil'

In [225]:
Ms = [2, 5, 10, 20, 50, 100, 200]
efcs = [10, 50, 100, 1000]
parameters_grid = list(product(Ms, efcs))
len(parameters_grid)

28

In [226]:
scores = {}

for space_name in ('cosinesimil', 'l2'):
    # Intitialize the library, specify the space, the type of the vector and add data points 
    index = nmslib.init(method='hnsw', space=space_name)
    index.addDataPointBatch(items_factors)
    
    for M, efC in tqdm(parameters_grid):
        index_time_params = {
            'M': M, 
            'indexThreadQty': num_threads, 
            'efConstruction': efC, 
        }
        index.createIndex(index_time_params, print_progress=True)

        # get all nearest neighbours for all the datapoint
        # using a pool of 2 threads to compute
        neighbours = index.knnQueryBatch(test_user_factors, k=10, num_threads=num_threads)
        neighbours = np.array(neighbours)

        recos_nmslib = neighbours[:, 0]
        score = calc_map(recos_nmslib)
        scores[(M, efC, space_name)] = score

  0%|                                                                                                                                                                                | 0/28 [00:00<?, ?it/s]
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
  4%|██████                                                                                                                                                                  | 1/28 [00:01<00:42,  1.58s/it]
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
  7%|████████████                                                                                                                                                            | 2/28 [00:03<00:45,  1.76s/it]
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|-

In [228]:
scores_s = pd.Series(scores).sort_values(ascending=False)
scores_s

200  100   l2             0.007944
     50    l2             0.007942
100  50    l2             0.007538
200  1000  l2             0.007465
     10    l2             0.007432
20   1000  l2             0.007304
50   10    l2             0.007272
100  1000  l2             0.006984
50   1000  l2             0.006937
     100   l2             0.006791
5    1000  l2             0.006515
20   100   l2             0.006502
10   100   l2             0.006209
100  100   l2             0.006172
     10    l2             0.006052
20   10    l2             0.005912
10   50    l2             0.005833
     1000  l2             0.005703
20   50    l2             0.004854
5    100   l2             0.004816
     50    l2             0.004705
50   50    l2             0.004234
10   10    l2             0.002996
2    1000  l2             0.002982
5    10    l2             0.002098
2    50    l2             0.001626
     10    l2             0.001542
     50    cosinesimil    0.001336
     1000  cosinesim

In [230]:
scores_s.head(1).index[0]

(200, 100, 'l2')

In [235]:
index = nmslib.init(method='hnsw', space='l2')
index.addDataPointBatch(items_factors)
for efC in [75, 125, 150]:
    index_time_params = {
        'M': 200, 
        'indexThreadQty': num_threads, 
        'efConstruction': efC, 
    }
    index.createIndex(index_time_params, print_progress=True)

    # get all nearest neighbours for all the datapoint
    # using a pool of 2 threads to compute
    neighbours = index.knnQueryBatch(test_user_factors, k=10, num_threads=num_threads)
    neighbours = np.array(neighbours)

    recos_nmslib = neighbours[:, 0]
    score = calc_map(recos_nmslib)
    scores[(M, efC, space_name)] = score


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [236]:
scores_s = pd.Series(scores).sort_values(ascending=False)
scores_s.head(10)

200  75    l2    0.007964
     100   l2    0.007944
     50    l2    0.007942
     150   l2    0.007936
     125   l2    0.007754
100  50    l2    0.007538
200  1000  l2    0.007465
     10    l2    0.007432
20   1000  l2    0.007304
50   10    l2    0.007272
dtype: float64

In [244]:
for space_name in tqdm(('l1', 'linf', 'angulardist', 'kldivfast')):
    index = nmslib.init(method='hnsw', space=space_name)
    index.addDataPointBatch(items_factors)
    index_time_params = {
        'M': 20, 
        'indexThreadQty': num_threads, 
        'efConstruction': 100, 
    }
    index.createIndex(index_time_params, print_progress=True)

    # get all nearest neighbours for all the datapoint
    # using a pool of 2 threads to compute
    neighbours = index.knnQueryBatch(test_user_factors, k=10, num_threads=num_threads)
    neighbours = np.array(neighbours)

    recos_nmslib = neighbours[:, 0]
    score = calc_map(recos_nmslib)
    scores[(M, efC, space_name)] = score


  0%|                                                                                                                                                                                 | 0/4 [00:00<?, ?it/s]
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

 25%|██████████████████████████████████████████▎                                                                                                                              | 1/4 [00:03<00:10,  3.37s/it]
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

 50%|████████████████████████████████████████████████████████████████████████████████████▌                                                                                    | 2/4 [00:06<00:06,  3.32s/it]
0%   10   20   30   40   50   60   70   80   90   100%
|----|---

In [256]:
scores_s = pd.Series(scores).sort_values(ascending=False)
scores_s.head(20)

200  75    l2    0.007964
     100   l2    0.007944
     50    l2    0.007942
     150   l2    0.007936
     125   l2    0.007754
100  50    l2    0.007538
200  1000  l2    0.007465
     10    l2    0.007432
20   1000  l2    0.007304
50   10    l2    0.007272
100  1000  l2    0.006984
50   1000  l2    0.006937
     100   l2    0.006791
5    1000  l2    0.006515
20   100   l2    0.006502
10   100   l2    0.006209
100  100   l2    0.006172
     10    l2    0.006052
20   10    l2    0.005912
10   50    l2    0.005833
dtype: float64

In [30]:
index = nmslib.init(method='hnsw', space='l2')
index.addDataPointBatch(items_factors)
index.createIndex(index_time_params, print_progress=True)

# get all nearest neighbours for all the datapoint
# using a pool of 2 threads to compute
neighbours = index.knnQueryBatch(test_user_factors, k=10, num_threads=num_threads)
neighbours = np.array(neighbours)

recos_nmslib = neighbours[:, 0]
score = calc_map(recos_nmslib)
score


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


0.008977320289076765

In [41]:
neighbours = index.knnQueryBatch(test_user_factors, k=50, num_threads=num_threads)
neighbours = np.array(neighbours)
recos_nmslib = neighbours[:, 0]
score = calc_map(recos_nmslib[:, :10])
score

0.008977320289076765

In [59]:
recos_nmslib_filtered = filter_seen(recos_nmslib)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95326/95326 [00:11<00:00, 7976.32it/s]


In [60]:
calc_map(recos_nmslib_filtered)

0.009846691725402425

После фильтрации скор улучшился

## hnswlib

In [68]:
%%time
n_recs = 10
spaces = ('ip', 'l2', 'cosine')
scores_hn = {}
for space in spaces:
    ann_index = hnswlib.Index(space=space, dim=model.factors)
    ann_index.init_index(max_elements=items_factors.shape[0])
    ann_index.add_items(items_factors)
    reco_ann, _ = ann_index.knn_query(test_user_factors, n_recs)
    score = calc_map(reco_ann)
    scores_hn[space] = score

In [69]:
pd.Series(scores_hn)

ip        0.016576
l2        0.004919
cosine    0.000761
dtype: float64

In [70]:
ann_index = hnswlib.Index(space='ip', dim=model.factors)
ann_index.init_index(max_elements=items_factors.shape[0])
ann_index.add_items(items_factors)
reco_ann, _ = ann_index.knn_query(test_user_factors, 50)

In [71]:
reco_ann_filtered = filter_seen(reco_ann)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95326/95326 [00:12<00:00, 7361.39it/s]


In [72]:
calc_map(reco_ann_filtered)

0.026263140193145944

После фильтрации скор значительно улучшился и пока это лучший скор

## Annoy

In [79]:
from annoy import AnnoyIndex

In [80]:
f = model.factors  # Length of item vector that will be indexed
t = AnnoyIndex(f, 'angular')

In [81]:
len(items_factors)

16519

In [88]:
spaces = ("angular", "euclidean", "manhattan", "hamming", "dot")
n_trees_list = np.arange(10, 300, 50)
grid = list(product(spaces, n_trees_list))
len(grid)

30

In [90]:
scores_annoy = {}
for space, n_trees in tqdm(grid):
    model = AnnoyIndex(f, space)
    
    for item_id, item_vec in enumerate(items_factors):
        model.add_item(item_id, item_vec)

    model.build(n_trees) # 10 trees
    annoy_recoms = np.array([model.get_nns_by_vector(user_vec, 10) for user_vec in test_user_factors])
    score = calc_map(annoy_recoms)
    scores_annoy[(space, n_trees)] = score

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [14:38<00:00, 29.29s/it]


In [91]:
pd.Series(scores_annoy).sort_values(ascending=False)

dot        260    0.011091
           210    0.010124
           110    0.010036
           160    0.009897
euclidean  260    0.007668
           210    0.007648
           160    0.007572
manhattan  260    0.007483
euclidean  110    0.007475
manhattan  210    0.007467
           160    0.007449
           110    0.007289
euclidean  60     0.007159
manhattan  60     0.007089
dot        60     0.006823
euclidean  10     0.005175
manhattan  10     0.004971
dot        10     0.003412
hamming    210    0.000892
           260    0.000891
angular    10     0.000880
           60     0.000773
           110    0.000772
           260    0.000728
           160    0.000725
           210    0.000720
hamming    10     0.000575
           110    0.000034
           160    0.000029
           60     0.000017
dtype: float64

In [92]:
space = 'dot'
n_trees = 400
model = AnnoyIndex(f, space)
    
for item_id, item_vec in enumerate(items_factors):
    model.add_item(item_id, item_vec)

model.build(n_trees) # 10 trees
annoy_recoms = np.array([model.get_nns_by_vector(user_vec, 10) for user_vec in test_user_factors])
score = calc_map(annoy_recoms)
scores_annoy[(space, n_trees)] = score

In [93]:
score

0.012268803798480167

In [94]:
%%time
space = 'dot'
n_trees = 600
model = AnnoyIndex(f, space)
    
for item_id, item_vec in enumerate(items_factors):
    model.add_item(item_id, item_vec)

model.build(n_trees) # 10 trees
annoy_recoms = np.array([model.get_nns_by_vector(user_vec, 10) for user_vec in test_user_factors])
score = calc_map(annoy_recoms)
scores_annoy[(space, n_trees)] = score

CPU times: user 2min 15s, sys: 2.14 s, total: 2min 17s
Wall time: 2min 9s


In [95]:
score

0.012624868765419536

In [97]:
annoy_recoms = np.array([model.get_nns_by_vector(user_vec, 50) for user_vec in test_user_factors])
calc_map(annoy_recoms[:, :10])

 50%|█████████████████████████████████████████████████████████████████████████████████▏                                                                               | 48105/95326 [31:08<30:34, 25.74it/s]


0.014349904560208696

In [100]:
annoy_recoms_filtered = filter_seen(annoy_recoms)
calc_map(annoy_recoms_filtered)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 95326/95326 [00:11<00:00, 8518.69it/s]


0.023249797626782896

Скор после фильтрации также улучшился

# Промежуточный итог

Лучшей на отложенной выборке получилась модель `hnswlib` с метрикой `ip`, хотя в целом у всех моделей скор достаточно низкий. Сделаю сабмит с этой моделью

# Load sample submission

In [102]:
submission_sample = pd.read_csv('data_full/sample_submission.csv')

In [103]:
cold_users = submission_sample.user_id[~submission_sample.user_id.isin(interactions.user_id)]

In [104]:
warm_users = submission_sample.user_id[submission_sample.user_id.isin(interactions.user_id)]

In [105]:
(cold_users.shape[0] + warm_users.shape[0]) / submission_sample.shape[0]

1.0

# Train final models

In [106]:
ui_csr = csr_matrix(
    (
        np.ones(len(interactions), dtype=np.float32),
        (
            interactions["user_id"].values,
            interactions["item_id"].values,
        ),
    ),
)

In [107]:
iu_csr = ui_csr.T.tocsr(copy=False)  # В версии implicit==0.4.4 матрицу нужно транспонировать

In [108]:
%%time
model = AlternatingLeastSquares(factors=32, iterations=10, num_threads=12)  # Если есть GPU, можно на нем
model.fit(iu_csr, show_progress=True)

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 2min 17s, sys: 15.5 s, total: 2min 32s
Wall time: 1min 23s


In [21]:
users_factors = model.user_factors
items_factors = model.item_factors

In [109]:
warm_user_factors = users_factors[warm_users]

In [110]:
ann_index = hnswlib.Index(space='ip', dim=model.factors)
ann_index.init_index(max_elements=items_factors.shape[0])
ann_index.add_items(items_factors)
reco_ann, _ = ann_index.knn_query(warm_user_factors, 50)

In [112]:
seen_items = interactions.groupby('user_id')['item_id']

In [127]:
reco_ann_filtered = filter_seen(reco_ann, warm_users)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128170/128170 [00:17<00:00, 7359.90it/s]


# Make submission

In [131]:
def make_df(recos, user_ids, n_recs=10):
    reco_df = pd.DataFrame(
        {
            Columns.User: np.repeat(user_ids, n_recs),
            Columns.Item: np.asarray(recos).ravel(),
            Columns.Rank: np.tile(np.arange(n_recs) + 1, len(user_ids)),
        }
    )
    return reco_df

In [134]:
def make_submission(recoms):
    most_popular = recoms.item_id.value_counts().index[:10].tolist() # для холодных юзеров
    submission = pd.Series(
        [most_popular] * submission_sample.shape[0], 
        index=submission_sample['user_id'],
        name='item_id'
    )
    sub = recoms[['user_id', 'item_id']].groupby('user_id').agg(lambda x: list(x))
    submission.loc[sub.index] = sub.item_id
    return submission.reset_index()

In [133]:
reco_df = make_df(reco_ann_filtered, warm_users, 10)

In [137]:
sub = make_submission(reco_df.astype(int))
sub.to_csv(f'submissions/hnswlib_filter_seen.csv', index=False)

In [216]:
! head -5 data_full/sample_submission.csv

user_id,item_id
3,"[9728, 15297, 10440, 14488, 13865, 12192, 341, 4151, 3734, 512]"
11,"[9728, 15297, 10440, 14488, 13865, 12192, 341, 4151, 3734, 512]"
29,"[9728, 15297, 10440, 14488, 13865, 12192, 341, 4151, 3734, 512]"
30,"[9728, 15297, 10440, 14488, 13865, 12192, 341, 4151, 3734, 512]"


In [138]:
! head -5 submissions/hnswlib_filter_seen.csv

user_id,item_id
3,"[14317, 6192, 6162, 15464, 7417, 6455, 5693, 11237, 14899, 12356]"
11,"[4457, 14317, 4436, 8636, 15464, 12360, 5693, 6455, 2720, 7793]"
29,"[947, 1819, 4191, 6162, 4111, 4879, 4813, 4405, 4296, 965]"
30,"[14317, 6162, 4436, 5693, 10240, 15464, 12623, 1785, 512, 6455]"
